# Introduction


In [ ]:
!ls /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL | wc -l
!ls /kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA | wc -l

!ls /kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL | wc -l
!ls /kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA | wc -l

!ls /kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL | wc -l
!ls /kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA | wc -l

!rm -rf /kaggle/working/chest_xray/

!mkdir -p /kaggle/working/chest_xray/train/NORMAL
!mkdir -p /kaggle/working/chest_xray/train/PNEUMONIA
!mkdir -p /kaggle/working/chest_xray/test/NORMAL
!mkdir -p /kaggle/working/chest_xray/test/PNEUMONIA
!mkdir -p /kaggle/working/chest_xray/val/NORMAL
!mkdir -p /kaggle/working/chest_xray/val/PNEUMONIA
!ls /kaggle/working/chest_xray/

!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/* | head -1341 | xargs cp -t /kaggle/working/chest_xray/train/NORMAL
!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/* | head -3875 | xargs cp -t /kaggle/working/chest_xray/train/PNEUMONIA

!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL/* | tail -184 | xargs cp -t /kaggle/working/chest_xray/val/NORMAL
!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/* | tail -262 | xargs cp -t /kaggle/working/chest_xray/val/PNEUMONIA

!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL/* | head -50 | xargs cp -t /kaggle/working/chest_xray/test/NORMAL
!ls -rt /kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/* | head -128 | xargs cp -t /kaggle/working/chest_xray/test/PNEUMONIA


# File layout

In [ ]:
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/train| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'
!ls /kaggle/working/chest_xray/train/NORMAL | wc -l
!ls /kaggle/working/chest_xray/train/PNEUMONIA | wc -l
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/val| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'
!ls /kaggle/working/chest_xray/val/NORMAL | wc -l
!ls /kaggle/working/chest_xray/val/PNEUMONIA | wc -l
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/test| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'
!ls /kaggle/working/chest_xray/test/NORMAL | wc -l
!ls /kaggle/working/chest_xray/test/PNEUMONIA | wc -l

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd

In [ ]:
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/train| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/val| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'
!ls -R /kaggle/input/chest-xray-pneumonia/chest_xray/test| grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//──/g' -e 's/─/├/' -e '$s/├/└/'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_image_gen = ImageDataGenerator(
    rotation_range=20, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, 
    rescale=1./255)
val_image_gen = ImageDataGenerator(rescale=1./255)
test_image_gen = ImageDataGenerator(rescale=1./255)

ds_train = train_image_gen.flow_from_directory('/kaggle/working/chest_xray/train', 
    class_mode='binary', color_mode="grayscale", batch_size=32, target_size=(256, 256))
ds_val = val_image_gen.flow_from_directory('/kaggle/working/chest_xray/val', 
    class_mode='binary', color_mode="grayscale", batch_size=32, target_size=(256, 256))
ds_test = test_image_gen.flow_from_directory('/kaggle/working/chest_xray/test', 
    class_mode='binary', color_mode="grayscale", batch_size=178, target_size=(256, 256))


In [ ]:
import matplotlib.pyplot as plt

class_names = {0:"NORMAL", 1:"PNEUMONIA"}
image_batch, label_batch = next(iter(ds_train))

for i in range(0, 16):
    ax = plt.subplot(4, 4, i + 1)
    plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top=1.5, wspace=0.2, hspace=0.2)
    plt.imshow(np.squeeze(image_batch[i], axis=2), cmap="gray")
    plt.title(class_names[int(label_batch[i])], fontsize=8)
    plt.axis("off")

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),    
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.BinaryCrossentropy(),
  metrics=['accuracy',
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall')])

In [ ]:
model.fit(
  ds_train,
  validation_data=ds_val,
  epochs=18
)

# Metrics

In [ ]:
from tensorflow.keras import backend
loss, acc, prec, rec = model.evaluate(ds_test)
print("Precision  ::", prec)
print("Recall     ::", rec)
print("F1     ::", 2*((prec*rec)/(prec+rec+backend.epsilon())))

In [ ]:
for image, label in iter(ds_test):
    print (label)
    for j in range(0,178):
        if j == 0:
            y_test = label[j].astype(int)
        else:
            y_test = np.append(label[j].astype(int), y_test)
    break    
print (len(y_test))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
y_pred1 = model.predict(ds_test)
y_pred = np.rint(y_pred1).astype(int)
y_pred = y_pred.flatten()
print (y_test)
print (y_pred)

In [ ]:
# Print f1, precision, and recall scores
print("Precision  ::", precision_score(y_test, y_pred))
print("Recall     ::", recall_score(y_test, y_pred))
print("F1 Score   ::", f1_score(y_test, y_pred))

# Visualizing the predictions

Incorrect predictions are highlighted in red color

In [ ]:
image_batch, label_batch = next(iter(ds_test))

for i in range(0, 25):
    ax = plt.subplot(5, 5, i + 1)
    plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top=1.5, wspace=0.2, hspace=0.2)
    if int(y_pred[i]) == int(y_test[i]):
        plt.imshow(np.squeeze(image_batch[i], axis=2), cmap="gray")
        plt.title(class_names[int(y_pred[i])], fontsize=8, color="green")
    else:
        plt.imshow(np.squeeze(image_batch[i], axis=2), cmap="gray")
        plt.title(class_names[int(y_pred[i])], fontsize=8, color="red")
    plt.axis("off")